# Keras model for Gesture Detection

## Defining transformers  

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
#from category_encoders.one_hot import OneHotEncoder
#from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Imputer
#from category_encoders.ordinal import OrdinalEncoder
from __future__ import print_function
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.api.types import is_numeric_dtype
import warnings
warnings.filterwarnings("ignore")
import ipytest.magics
import pytest
# set the file name (required)
__file__ = 'drone_pos_model.ipynb'

In [2]:
class Shuffler(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        x=x.loc[np.random.permutation(x.index)]
        
        return x
############################################################################################
class XCentralizer(BaseEstimator, TransformerMixin):
    
    def __init__(self, x_columns):
        self.x_columns = x_columns
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        shift=x[["rightShoulder_x","leftShoulder_x","leftHip_x","rightHip_x"]].sum(axis=1)/4
        for col in self.x_columns:
            x[col] = x[col] - shift
        return x
############################################################################################
    
class YCentralizer(BaseEstimator, TransformerMixin):
    
    def __init__(self, y_columns):
        self.y_columns = y_columns
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        shift=x[["rightShoulder_y","leftShoulder_y","leftHip_y","rightHip_y"]].sum(axis=1)/4
        for col in list(set(self.y_columns)-set(["label"])):
            x[col] = x[col] - shift
        return x
############################################################################################

class YScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        shoulder_y = x[["rightShoulder_y","leftShoulder_y"]].sum(axis=1)/2
        hip_y = x[["leftHip_y","rightHip_y"]].sum(axis=1)/2
        y_dist = hip_y - shoulder_y
        
        for col in list(set(x.columns)-set(["label"])):
            x[col] /= y_dist
        return x
###############################################################################################
def baseNameSelector(baseName, noIdx =8):
    names = []
    for i in range(noIdx):
        names.append(baseName + "_" + str(i))
    return names    
    

## Data inspecting

In [3]:
! pwd

/Users/lsafari/drone_steering/models


In [5]:
path = "video_001_gesture_steptime100_checksum8160_seqlength800"
df = pd.read_csv("/Users/lsafari/drone_steering/data/"+ path + ".csv",low_memory=False)
print(df.shape)
print('labels:' , df['label'].unique())
df[:5]

(179, 129)
labels: [1 3 4 2]


,leftShoulder_x_0,leftShoulder_y_0,rightShoulder_x_0,rightShoulder_y_0,leftElbow_x_0,leftElbow_y_0,rightElbow_x_0,rightElbow_y_0,leftWrist_x_0,leftWrist_y_0,...,rightElbow_y_7,leftWrist_x_7,leftWrist_y_7,rightWrist_x_7,rightWrist_y_7,leftHip_x_7,leftHip_y_7,rightHip_x_7,rightHip_y_7,label
0,0.4925,0.18625,0.40000,0.18250,0.50750,0.25875,0.34000,0.19750,0.50750,0.33875,...,0.19500,0.50375,0.33875,0.26000,0.17750,0.46500,0.34250,0.40750,0.34125,1
1,0.4900,0.19000,0.40250,0.18250,0.50250,0.26250,0.33500,0.20000,0.50125,0.33375,...,0.19875,0.50375,0.34000,0.44500,0.34250,0.46125,0.33375,0.40750,0.33875,1
2,0.4900,0.19000,0.40500,0.18125,0.50625,0.25750,0.33125,0.20250,0.50000,0.33000,...,0.20000,0.50000,0.33625,0.44875,0.33625,0.46375,0.34125,0.40500,0.34000,1
3,0.4900,0.18875,0.40250,0.18000,0.50500,0.25875,0.33625,0.19875,0.50125,0.33250,...,0.20500,0.50250,0.34000,0.27375,0.18375,0.46250,0.34125,0.40500,0.33750,1
4,0.4925,0.18875,0.40125,0.18250,0.50375,0.25750,0.33625,0.19250,0.49875,0.33375,...,0.20375,0.49875,0.33625,0.29125,0.19125,0.46000,0.33625,0.40875,0.33500,1


In [8]:
df[baseNameSelector("leftWrist_y")].head()

,leftWrist_y_0,leftWrist_y_1,leftWrist_y_2,leftWrist_y_3,leftWrist_y_4,leftWrist_y_5,leftWrist_y_6,leftWrist_y_7
0,0.33875,0.33375,0.33000,0.33250,0.33375,0.34000,0.33625,0.33875
1,0.33375,0.33000,0.33250,0.33375,0.34000,0.33625,0.33875,0.34000
2,0.33000,0.33250,0.33375,0.34000,0.33625,0.33875,0.34000,0.33625
3,0.33250,0.33375,0.34000,0.33625,0.33875,0.34000,0.33625,0.34000
4,0.33375,0.34000,0.33625,0.33875,0.34000,0.33625,0.34000,0.33625


In [9]:
import os 
fname = "/Users/lsafari/drone_steering/data/"+ path + ".csv"
f = open(fname)
data = f.read()
f.close()

lines = data.split("\n")
header = lines[0].split(",")
lines = lines[1:]

print(header)
print(len(header))
print(len(lines))
print(lines[0])
print(type(lines[0]))

['leftShoulder_x_0', 'leftShoulder_y_0', 'rightShoulder_x_0', 'rightShoulder_y_0', 'leftElbow_x_0', 'leftElbow_y_0', 'rightElbow_x_0', 'rightElbow_y_0', 'leftWrist_x_0', 'leftWrist_y_0', 'rightWrist_x_0', 'rightWrist_y_0', 'leftHip_x_0', 'leftHip_y_0', 'rightHip_x_0', 'rightHip_y_0', 'leftShoulder_x_1', 'leftShoulder_y_1', 'rightShoulder_x_1', 'rightShoulder_y_1', 'leftElbow_x_1', 'leftElbow_y_1', 'rightElbow_x_1', 'rightElbow_y_1', 'leftWrist_x_1', 'leftWrist_y_1', 'rightWrist_x_1', 'rightWrist_y_1', 'leftHip_x_1', 'leftHip_y_1', 'rightHip_x_1', 'rightHip_y_1', 'leftShoulder_x_2', 'leftShoulder_y_2', 'rightShoulder_x_2', 'rightShoulder_y_2', 'leftElbow_x_2', 'leftElbow_y_2', 'rightElbow_x_2', 'rightElbow_y_2', 'leftWrist_x_2', 'leftWrist_y_2', 'rightWrist_x_2', 'rightWrist_y_2', 'leftHip_x_2', 'leftHip_y_2', 'rightHip_x_2', 'rightHip_y_2', 'leftShoulder_x_3', 'leftShoulder_y_3', 'rightShoulder_x_3', 'rightShoulder_y_3', 'leftElbow_x_3', 'leftElbow_y_3', 'rightElbow_x_3', 'rightElbow_y

In [ ]:
#lines[0].split(",")[:-1]

In [10]:
import numpy as np

float_data = np.zeros((len(lines), len(header) - 1)) # empty array
print(float_data.shape) # (sample, features)

for i, line in enumerate(lines): # enumerate indexes the lines
    values = [float(x) for x in line.split(",")[:-1]] #
    float_data[i, :] = values
    
print(float_data.shape)

(179, 128)
(179, 128)


In [11]:
print(float_data[0].shape)

(128,)


In [ ]:
mean = float_data[:178].mean(axis=0)
float_data -= mean
std = float_data[:178].std(axis=0)
float_data /= std

In [15]:
float_data.shape

(179, 128)

In [ ]:
#float_data[0]

In [24]:
i, lookback, min_index, len(float_data), max_index

(178, 100, 0, 179, 168)

In [26]:
float_data.shape[-1]

128

In [23]:
lookback = 100# 5 days in the past in a fixed t
step = 1 #one sample per hour =time step
delay = 10 #target 24h in the future
batch_size=50
min_index=0
max_index=len(float_data) - delay - 1
def generator( # is a object
    data,
    lookback, delay,
    min_index, max_index,
    shuffle=False, # val we dont shaffel
    batch_size=batch_size,
    step=6):

    if max_index is None: # none= does not exist
        max_index = len(data) - delay - 1
    i = min_index + lookback # TODO redundant?
    while True: # means do 

        if shuffle:
            rows = np.random.randint(
                min_index + lookback,
                max_index,
                batch_size
            )
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]

        yield samples, targets
        
next(generator(float_data, lookback, delay, 0, max_index))[1].shape
#(sample, feature, )   [1]=output, [0]=input
next(generator(float_data, lookback, delay, 0, max_index))[0].shape

ValueError: could not broadcast input array from shape (17,128) into shape (16,128)

# Training Model

# Model Architecture

In [25]:
from keras import models, layers
from keras.models import Model
from keras.layers import Input, Dense
from keras import optimizers, losses, metrics


#default vaues
#activation="relu"
#optimizer="adam"
lr=0.01
#momentum=0
#creat model
model = models.Sequential()
model.add(layers.Dense(
        20, 
        activation="relu", 
        input_shape=(16, )))
model.add(layers.Dense(15, activation="relu"))
model.add(layers.Dense(5, activation="softmax")) #is a fast rectifier
model.summary()   

model.compile(
optimizer=optimizers.RMSprop(lr=0.01),
loss=losses.categorical_crossentropy,
metrics=["accuracy"] 
)
    

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                340       
_________________________________________________________________
dense_2 (Dense)              (None, 15)                315       
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 80        
Total params: 735
Trainable params: 735
Non-trainable params: 0
_________________________________________________________________
